<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a </span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
 
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.dummy import DummyRegressor
 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,  make_scorer
 
from pymystem3 import Mystem
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler

 
from nltk.corpus import stopwords, wordnet
from nltk.corpus import stopwords as nltk_stopwords
import nltk
nltk.download('stopwords') 
#from sklearn.feature_extraction.text import CountVectorizer 
 
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')
 
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
data

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


In [4]:
df = data.copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


проведем очистку текста

In [5]:
m = WordNetLemmatizer()

def lemmatize_text(text):    
    text = text.lower()
    word_list = nltk.word_tokenize(text)
    lemm_text = ' '.join([m.lemmatize(w) for w in word_list])
    #lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

проведем лемматизацию

In [6]:
df['text'] = df['text'].apply(lemmatize_text)
target = df['toxic']
features = df.drop(['toxic'], axis=1)

разделим данные на тренировочные и тестовые

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features, 
                                                                              target, 
                                                                              test_size=0.4, 
                                                                              random_state=42)

In [8]:
stopwords = set(nltk_stopwords.words('english')) # "стопслова"

count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

посчитаем Tf-IDF

In [9]:
features_train = count_tf_idf.fit_transform(features_train['text'].values)

features_test = count_tf_idf.transform(features_test['text'].values)

In [10]:
features_train

<95742x119712 sparse matrix of type '<class 'numpy.float64'>'
	with 2605564 stored elements in Compressed Sparse Row format>

## Обучение

Подготовили данные, теперь обучим модель "Дерева решений" для классификации

In [32]:
tree = DecisionTreeClassifier(random_state = 123)
param_grid = {
   'decisiontreeclassifier__criterion':['gini', 'entropy'],        
   'decisiontreeclassifier__max_depth':[4,6,10,12]
}
pip_gs = make_pipeline(TfidfVectorizer(stop_words=stopwords), tree) #with_mean=False
tree_gs = GridSearchCV(pip_gs,
                       param_grid=param_grid,
                       cv=3, scoring='f1',
                       verbose=True)
tree_gs.fit(features_train, target_train)
tree_gs.best_score_, tree_gs.best_params_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 341, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 303, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/opt/conda/lib/python3.9/site-packages/joblib/memory.py", line 349, in __call__
    return self.func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 754, in _fit_transform_one
    res = transformer.fit_transform(X, y, **fit_params)
  File "/opt/conda/lib/python3.

AttributeError: lower not found

Результат 0,6 не высокий и не удовлетворяет нашему условию >0,75

Проверим работу модели Логистической регрессии

In [130]:
lr = LogisticRegression(class_weight='balanced', random_state=123)
params = {
    'logisticregression__C': (0.1, 1, 10),
    'logisticregression__penalty': ('l1', 'l2')
}
pip_lr = make_pipeline(StandardScaler(with_mean=False), lr)
lr_gs = GridSearchCV(pip_lr, param_grid=params, cv=3, scoring='f1')

lr_gs.fit(features_train, target_train)
lr_gs.score(features_test, target_test)

/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/pipeline.py", line 346, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warni

0.643044816455221

In [91]:
lr_gs.best_params_

{'C': 10, 'penalty': 'l2'}

Показатель F1 = 0,77 что вполне удовлетворяет поставленной задачи

Проверим работу модели Случайного леса

In [133]:
rf=RandomForestClassifier(class_weight='balanced', random_state=123)
params = {
    'randomforestclassifier__n_estimators': [10, 50, 100],
    'randomforestclassifier__max_depth': [10, 12, 16, 20, 26],
    'randomforestclassifier__random_state': [123]
}
pip_rf = make_pipeline(StandardScaler(with_mean=False), rf)
rf_gs = GridSearchCV(pip_rf, param_grid=params, cv=5, scoring='f1').fit(features_train, target_train)
rf_gs.best_score_, rf_gs.best_params_

(0.42183002638225997,
 {'randomforestclassifier__max_depth': 26,
  'randomforestclassifier__n_estimators': 100,
  'randomforestclassifier__random_state': 123})

In [132]:
pip_rf.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'standardscaler', 'randomforestclassifier', 'standardscaler__copy', 'standardscaler__with_mean', 'standardscaler__with_std', 'randomforestclassifier__bootstrap', 'randomforestclassifier__ccp_alpha', 'randomforestclassifier__class_weight', 'randomforestclassifier__criterion', 'randomforestclassifier__max_depth', 'randomforestclassifier__max_features', 'randomforestclassifier__max_leaf_nodes', 'randomforestclassifier__max_samples', 'randomforestclassifier__min_impurity_decrease', 'randomforestclassifier__min_impurity_split', 'randomforestclassifier__min_samples_leaf', 'randomforestclassifier__min_samples_split', 'randomforestclassifier__min_weight_fraction_leaf', 'randomforestclassifier__n_estimators', 'randomforestclassifier__n_jobs', 'randomforestclassifier__oob_score', 'randomforestclassifier__random_state', 'randomforestclassifier__verbose', 'randomforestclassifier__warm_start'])

In [134]:
rf_gs.fit(features_train, target_train)
rf_gs.score(features_test, target_test)

0.4329589824095319

In [92]:
result=pd.DataFrame([{'Точночть F1':'0,770', 'параметры':'C=10, penalty=l2'}
                     , {'Точночть F1':'0,675', 'параметры':'criterion=gini, max_depth=31'}
                    , {'Точночть F1':'0,425', 'параметры':'max_depth=26, n_estimators=100'}],
                    index=['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier'])
display(result)

,Точночть F1,параметры
LogisticRegression,"0,770","C=10, penalty=l2"
DecisionTreeClassifier,"0,675","criterion=gini, max_depth=31"
RandomForestClassifier,"0,425","max_depth=26, n_estimators=100"


## Выводы

**Вывод:**
- провели очистку данных и лемматизацию
- убрали "стопслова" и подсчитали Tf-IDF текстов
- разделили данные на тренировочные и тестовые в соотношении 60% и 40%
- проверили работу 3-х моделей: "Логистическая регрессия", "Дерево решений" и "Случайный лес"
Модель `LogisticRegression` показала лучший результат качества.
С параметрами: 'C': 10, 'penalty': 'l2'

эта модель будет давать наиболее точные предсказания позитивных и негативных комментариев.